1.Задание 1
- построить матрицу.
- Заменить 0 на 0.01, после
- нормализовать ее
- и умножить на вектор [х]0ю2..... несколько раз
- топ 10 смапить с матрицы
- армстрнг 10 топ

2.hists
- умножить матрицу на транспанированную
- найти собственный вектор
- hub score
- обратое действие трансп на обычную и найти собственный вектор
- ha

Ссылка на данные: https://cloud.mail.ru/public/2yaK/q4DDyYf3R

In [1]:
import os
import urllib
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm_notebook as tn

In [2]:
filenames = os.listdir("raw")

In [3]:
len(filenames)

2379

In [4]:
topics = {}
for i, file in enumerate(filenames):
    topics[file[:-5]] = i

In [5]:
matrix = np.zeros((len(filenames), len(filenames)))

In [6]:
for file in tn(filenames):
    with open(f"raw/{file}", encoding='utf-8') as f:
        cur_topic = file[:-5]
        bs = BeautifulSoup(f.read(), "html.parser")
        all_a = bs.find_all("a")
        for link in all_a:
            if not 'href' in link.attrs:
                continue
            href = urllib.parse.unquote(link['href'])
            if href.startswith("/wiki/") and href[6:] in topics:
                matrix[topics[cur_topic]][topics[href[6:]]] = 1

In [7]:
for i in matrix:
    for j in range(len(i)):
        if i[j] == 0:
            i[j] = 0.001

In [8]:
row_sums = matrix.sum(axis=1)

In [9]:
norm_matrix = matrix / row_sums[:, np.newaxis]

In [10]:
vector = np.array([0.2]*len(filenames))

In [11]:
for _ in range(100):
    vector = norm_matrix.transpose().dot(vector)

In [12]:
np.take(vector, [1, 2])

array([0.90675362, 0.75596967])

In [13]:
rank = {}

In [14]:
for i, value in enumerate(vector):
    rank[i] = value

In [15]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
        
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{filenames[index][:-5]}: {value}")

In [16]:
page_rank(rank=rank)

Английский_язык: 6.7102441063494815
США: 4.666826841990532
Франция: 3.891332065960341
Великобритания: 3.7553791117132835
Германия: 3.045402505156282
Соединённые_Штаты_Америки: 2.982485368742146
Италия: 2.411497480410388
СССР: 2.385908911034202
Викисклад: 2.308323432805024
Латинский_язык: 2.305412331063708


In [17]:
page_rank(rank=rank, word='армстронг')

Армстронг,_Луи: 0.2191307392457279
Армстронг,_Нил_Олден: 0.08204167651131539
Армстронг,_Нил: 0.07772736651558697
Армстронг,_Билли_Джо: 0.06033081219855766
Армстронг,_Алан: 0.05467384689837547
Армстронг_(фамилия): 0.054339929130478144
Армстронг,_Эдвин: 0.048724294245225605
Армстронг,_Лэнс: 0.04765582655907429
Билли_Джо_Армстронг: 0.04497463456555407
Армстронг,_Крэйг: 0.039019336753872536
